In [35]:
import math
import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import pickle
import random
import math
import time
import pprint
from constants import DIST_METRIC_INDEX

In [36]:
def init_memmap_embs(
    embs_memory_loc: str, dataset_size: int, emd_size: int = 512, dtype: str = "float32"
) -> np.memmap:
    """
    Initializes a memory-mapped NumPy array to read embeddings of examples.

    Args:
        embs_memory_loc (str): Path to the memory-mapped file.
        dataset_size (int): Size of the dataset.
        emd_size (int): Dimensionality of the embeddings.
        dtype (str): Data type of the embeddings.

    Returns:
        np.memmap: A memory-mapped NumPy array.
    """
    embs = np.memmap(
        embs_memory_loc, dtype=dtype, mode="r", shape=(dataset_size, emd_size)
    )
    return embs

In [41]:
class SemDeDup():

    def __init__(self, args):
        self.args = args
        random.seed(args["seed"])

    def contains_duplicates(self, arr):
        return len(np.unique(arr)) != len(arr)

    def semdedup(self, cluster, cluster_reps, device):
        st = time.time()
        ## -- compute pairwise cos sim between cluster items, then replace to diagonal with zeros to ignore self similarity
        cluster_reps.to(device)
        pair_w_sim_matrix = cluster_reps @ (cluster_reps.T)
        del cluster_reps
        pair_w_sim_matrix.fill_diagonal_(0.0)
        assert pair_w_sim_matrix.shape[0] == pair_w_sim_matrix.shape[1]

        # TODO: what are you doung with image_urls?
        ## -- get paths to cluster i images
        image_urls = cluster[:, 0]

        ## -- make sure all the paths are unique this ensure that the duplicates are really stored many time times on memory
        assert not self.contains_duplicates(image_urls)

        ## -- We need upper tringular matrix because (1)we don't need to look at self sim (always=1) (2)we need the compinations not permutations
        triu_sim_mat = torch.triu(pair_w_sim_matrix, diagonal=1)

        ## -- if the max sim between one example and any other example is > 1-eps, remove this example
        M = torch.max(triu_sim_mat, dim=0)[0].cpu()
        print(f"Step time: {time.time()-st}(s)")

        return M

    def process_clusters(self, start_cluster: int, end_cluster: int):
        # print("SemDeDup params: ", self.args)
        st = time.time()

        embs = init_memmap_embs(
            self.args["embs_memory_loc"], self.args["dataset_size"], self.args["emd_size"]
        )

        step_time = []

        for cluster_id in tqdm(range(start_cluster, end_cluster)):
            step_st = time.time()

            df_file_loc = os.path.join(
                self.args["save_loc"], f"dataframes/cluster_{cluster_id}.pkl"
            )

            if os.path.exists(df_file_loc):  # and os.path.exists(dict_file_loc):
                print(f"{df_file_loc} exists, moving on")
                continue

            ## -- load cluster i representations
            cluster_i = np.load(
                os.path.join(
                    self.args["sorted_clusters_path"], f"cluster_{cluster_id}.npy"
                )
            )
            # 1) store cluster size
            cluster_size = cluster_i.shape[0]
            print("cluster_size: ", cluster_size)

            if cluster_size == 1:
                points_to_remove_df = pd.DataFrame()
                points_to_remove_df["indices"] = [0]
                for eps in self.args["eps_list"]:
                    ## We need to remove a point from the dataset when its pairwise similarity to other point is > 1-ebs
                    points_to_remove_df[f"eps={eps}"] = [False]
                if self.args["save_loc"] != "":
                    ## --save df
                    with open(df_file_loc, "wb") as file:
                        pickle.dump(points_to_remove_df, file)
                print("DONE cluster_id ", cluster_id)
                continue

            ## -- By default, we keep hard examples from groups
            clutser_items_indices = list(range(cluster_size))
            ## -- OR: shuffle cluster to keep random example from each group
            if self.args["which_to_keep"].lower() == "random":
                random.shuffle(clutser_items_indices)
                cluster_i = cluster_i[clutser_items_indices]
            ## -- OR: reverse cluster to keep easy examples
            if self.args["which_to_keep"].lower() == "easy":
                clutser_items_indices = clutser_items_indices[::-1]
                cluster_i = cluster_i[clutser_items_indices]

            ## -- indices for cluster items in the dataset
            cluster_ids = cluster_i[:, 1].astype("int32")
            cluster_reps = embs[cluster_ids]
            cluster_reps = torch.tensor(cluster_reps)

            M = self.semdedup(cluster_i, cluster_reps, self.args["device"])

            points_to_remove_df = pd.DataFrame()
            points_to_remove_df["indices"] = clutser_items_indices

            for eps in self.args["eps_list"]:
                ## -- 5) We need to remove a point from the dataset when its pairwise similarity to other point is > 1-ebs
                eps_points_to_remove = M > 1 - eps
                points_to_remove_df[f"eps={eps}"] = eps_points_to_remove

            if self.args["save_loc"] != "":
                ## --save df
                with open(df_file_loc, "wb") as file:
                    pickle.dump(points_to_remove_df, file)

            step_time.append(time.time() - step_st)
            print("DONE cluster: ", cluster_id)

        print(
            f"DONE in {((time.time()-st)/60):.2f} minutes, Average Step time {(sum(step_time)/len(step_time)):.2f}(s)"
        )
        return

    def call(self):
        pp = pprint.PrettyPrinter(indent=4)
        pp.pprint(self.args)
        start_cluster = 0
        end_cluster = 999
        self.process_clusters(start_cluster, end_cluster)


In [42]:
args = {
    "seed": 5,
    "embs_memory_loc": "/workspace/CS762_Project/SemDeDup/code_alpaca_results/emb_mmap.dat",
    "dataset_size": 2017,
    "emd_size": 1024,
    "save_loc": "/workspace/CS762_Project/SemDeDup/code_alpaca_results/save_location",
    "sorted_clusters_path": "/workspace/CS762_Project/SemDeDup/code_alpaca_results/sorted_clusters",
    "eps_list": [0.9, 0.95, 0.98],
    "which_to_keep": "random",
    "device": "cuda",
}

In [43]:
semdedup = SemDeDup(args)

In [44]:
semdedup.call()

{   'dataset_size': 2017,
    'device': 'cuda',
    'embs_memory_loc': '/workspace/CS762_Project/SemDeDup/code_alpaca_results/emb_mmap.dat',
    'emd_size': 1024,
    'eps_list': [0.9, 0.95, 0.98],
    'save_loc': '/workspace/CS762_Project/SemDeDup/code_alpaca_results/save_location',
    'seed': 5,
    'sorted_clusters_path': '/workspace/CS762_Project/SemDeDup/code_alpaca_results/sorted_clusters',
    'which_to_keep': 'random'}


  2%|▏         | 21/999 [00:00<00:08, 111.01it/s]

/workspace/CS762_Project/SemDeDup/code_alpaca_results/save_location/dataframes/cluster_0.pkl exists, moving on
/workspace/CS762_Project/SemDeDup/code_alpaca_results/save_location/dataframes/cluster_1.pkl exists, moving on
/workspace/CS762_Project/SemDeDup/code_alpaca_results/save_location/dataframes/cluster_2.pkl exists, moving on
/workspace/CS762_Project/SemDeDup/code_alpaca_results/save_location/dataframes/cluster_3.pkl exists, moving on
/workspace/CS762_Project/SemDeDup/code_alpaca_results/save_location/dataframes/cluster_4.pkl exists, moving on
cluster_size:  3
Step time: 0.015244483947753906(s)
DONE cluster:  5
cluster_size:  1
DONE cluster_id  6
cluster_size:  1
DONE cluster_id  7
cluster_size:  2
Step time: 0.00019025802612304688(s)
DONE cluster:  8
cluster_size:  2
Step time: 0.00018906593322753906(s)
DONE cluster:  9
cluster_size:  1
DONE cluster_id  10
cluster_size:  1
DONE cluster_id  11
cluster_size:  1
DONE cluster_id  12
cluster_size:  4
Step time: 0.00023555755615234375(

  3%|▎         | 33/999 [00:00<00:08, 110.36it/s]

Step time: 0.08930087089538574(s)
DONE cluster:  23
cluster_size:  3
Step time: 0.0002918243408203125(s)
DONE cluster:  24
cluster_size:  1
DONE cluster_id  25
cluster_size:  3
Step time: 0.00019168853759765625(s)
DONE cluster:  26
cluster_size:  1
DONE cluster_id  27
cluster_size:  1
DONE cluster_id  28
cluster_size:  1
DONE cluster_id  29
cluster_size:  1
DONE cluster_id  30
cluster_size:  1
DONE cluster_id  31
cluster_size:  2
Step time: 0.0011267662048339844(s)
DONE cluster:  32
cluster_size:  2
Step time: 0.0001933574676513672(s)
DONE cluster:  33
cluster_size:  2
Step time: 0.08681750297546387(s)
DONE cluster:  34
cluster_size:  1
DONE cluster_id  35
cluster_size:  4
Step time: 0.0002918243408203125(s)
DONE cluster:  36
cluster_size:  1
DONE cluster_id  37
cluster_size:  2
Step time: 0.08677029609680176(s)
DONE cluster:  38


...

cluster_size:  3
Step time: 0.00027060508728027344(s)
DONE cluster:  925
cluster_size:  3
Step time: 0.0001697540283203125(s)
DONE cluster:  926
cluster_size:  2
Step time: 0.07940793037414551(s)
DONE cluster:  927
cluster_size:  1
DONE cluster_id  928
cluster_size:  2
Step time: 0.00026917457580566406(s)
DONE cluster:  929
cluster_size:  2
Step time: 0.0001671314239501953(s)
DONE cluster:  930
cluster_size:  1
DONE cluster_id  931
cluster_size:  4
Step time: 0.00017881393432617188(s)
DONE cluster:  932
cluster_size:  2
Step time: 0.0001595020294189453(s)
DONE cluster:  933
cluster_size:  3
Step time: 0.0001652240753173828(s)
DONE cluster:  934
cluster_size:  2
Step time: 0.000164031982421875(s)
DONE cluster:  935
cluster_size:  1
DONE cluster_id  936
cluster_size:  3
Step time: 0.00016689300537109375(s)
DONE cluster:  937
cluster_size:  1
DONE cluster_id  938
cluster_size:  4
Step time: 0.0001735687255859375(s)
DONE cluster:  939
cluster_size:  1
DONE cluster_id  940
cluster_size:  4


 97%|█████████▋| 968/999 [00:11<00:00, 124.24it/s]

cluster_size:  1
DONE cluster_id  951
cluster_size:  3
Step time: 0.00017023086547851562(s)
DONE cluster:  952
cluster_size:  2
Step time: 0.0795447826385498(s)
DONE cluster:  953
cluster_size:  1
DONE cluster_id  954
cluster_size:  2
Step time: 0.0002586841583251953(s)
DONE cluster:  955
cluster_size:  1
DONE cluster_id  956
cluster_size:  2
Step time: 0.00017118453979492188(s)
DONE cluster:  957
cluster_size:  1
DONE cluster_id  958
cluster_size:  3
Step time: 0.0001735687255859375(s)
DONE cluster:  959
cluster_size:  2
Step time: 0.00016808509826660156(s)
DONE cluster:  960
cluster_size:  2
Step time: 0.00016307830810546875(s)
DONE cluster:  961
cluster_size:  2
Step time: 0.00016736984252929688(s)
DONE cluster:  962
cluster_size:  1
DONE cluster_id  963
cluster_size:  2
Step time: 0.00016927719116210938(s)
DONE cluster:  964
cluster_size:  1
DONE cluster_id  965
cluster_size:  2
Step time: 0.08025169372558594(s)
DONE cluster:  966
cluster_size:  3
Step time: 0.00025081634521484375(

100%|██████████| 999/999 [00:11<00:00, 84.03it/s] 

DONE cluster:  978
cluster_size:  2
Step time: 0.08025193214416504(s)
DONE cluster:  979
cluster_size:  1
DONE cluster_id  980
cluster_size:  2
Step time: 0.0002605915069580078(s)
DONE cluster:  981
cluster_size:  2
Step time: 0.00017023086547851562(s)
DONE cluster:  982
cluster_size:  2
Step time: 0.00016736984252929688(s)
DONE cluster:  983
cluster_size:  1
DONE cluster_id  984
cluster_size:  1
DONE cluster_id  985
cluster_size:  1
DONE cluster_id  986
cluster_size:  1
DONE cluster_id  987
cluster_size:  1
DONE cluster_id  988
cluster_size:  1
DONE cluster_id  989
cluster_size:  1
DONE cluster_id  990
cluster_size:  1
DONE cluster_id  991
cluster_size:  3
Step time: 0.0002853870391845703(s)
DONE cluster:  992
cluster_size:  3
Step time: 0.0801246166229248(s)
DONE cluster:  993
cluster_size:  4
Step time: 0.00023698806762695312(s)
DONE cluster:  994
cluster_size:  1
DONE cluster_id  995
cluster_size:  2
Step time: 0.0001780986785888672(s)
DONE cluster:  996
cluster_size:  2
Step time: